In [1]:
import os
import boto3
import re
import json
import pandas as pd
import numpy as np
import sagemaker
import scipy.sparse
from sagemaker import get_execution_role
from sagemaker.sklearn.model import SKLearnModel
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
region = boto3.Session().region_name

bucket = sagemaker.Session().default_bucket()

print(f"bucket: {bucket}")

bucket: sagemaker-us-east-2-034362035459


In [3]:
# Get a Sagemaker compatible role used by this notebook instance
role = get_execution_role()

In [4]:
role

'arn:aws:iam::034362035459:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole'

In [5]:
import sagemaker
sagemaker_session = sagemaker.Session()

train_input = sagemaker_session.upload_data('mydata')

In [6]:
from sagemaker.sklearn import SKLearn

sk_estimator = SKLearn(
    entry_point="train.py",
    role=role,
    instance_count=1,
    instance_type="ml.c5.xlarge",
    py_version="py3",
    framework_version="1.0-1",
    hyperparameters={"estimators": 20},
)

In [7]:
# Train the estimator
sk_estimator.fit({"train": train_input})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-08-28-02-53-05-735


2024-08-28 02:53:06 Starting - Starting the training job...
2024-08-28 02:53:27 Starting - Preparing the instances for training...
2024-08-28 02:53:53 Downloading - Downloading input data...
2024-08-28 02:54:13 Downloading - Downloading the training image...
2024-08-28 02:55:04 Training - Training image download completed. Training in progress.
2024-08-28 02:55:04 Uploading - Uploading generated training model2024-08-28 02:54:59,170 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-08-28 02:54:59,172 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-28 02:54:59,175 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-08-28 02:54:59,189 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-08-28 02:54:59,403 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-08-28 02:54:59,406 sagemaker-training-toolki

In [8]:
deployment = sk_estimator.deploy(initial_instance_count = 1, instance_type ='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2024-08-28-03-04-19-510
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2024-08-28-03-04-19-510
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2024-08-28-03-04-19-510


----------------------------------------------*

UnexpectedStatusException: Error hosting endpoint sagemaker-scikit-learn-2024-08-28-03-04-19-510: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html